# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Описание признаков

Features - Пол, Возраст, Зарплата, Члены семьи.
Target - Страховые выплаты

In [2]:
import pandas as pd
import numpy as np 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

## Загрузка данных

In [3]:
data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Если признаки умножают на обратимую матрицу, качество линейной регрессии не изменится. 

**Обоснование:** 
У нас есть ряд свойств:
    $$
(AB)^T=B^T A^T \tag 1
$$
$$
(AB)^{-1} = B^{-1} A^{-1} \tag 2
$$
$$
A A^{-1} = A^{-1} A = E \tag 3
$$
$$
AE = EA = A \tag 4
$$
$$
A(BC) = (AB)C \tag 5
$$
    
    
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
    
Для начала раскроем $(XP)^T$ используя свойство  $(AB)^T = B^T A^T$ 
    
$$
w' = (P^T X^T XP)^{-1} P^T X^T y
$$    
   
Далее раскроем   $(P^T(X^TX)P)^{-1}$, пусть тут $A=P^T$, $B=(X^TX)P)^{-1}$, тогда $(P^T(X^TX)P)^{-1} = ((X^TX)P)^{-1} P^{T^{-1}}$
    
$$
w' = (X^T XP)^{-1} P^{T^{-1}} P^T X^T y
$$  
    
$P^{T^{-1}} P^T$ - сократим
    
$$
w' = (X^T XP)^{-1} X^T y
$$
    
Далее раскроем $(X^T XP)^{-1}$, пусть тут $A=(X^TX)$, $B=P$
    
    
$$
w' = P^{-1}(X^T X)^{-1} X^T y = P^{-1}w
$$    

$$
a' = XPP^{-1}w = Xw = a
$$

ч.т.д    

In [4]:
# Исходные данные (признаки)
features = np.array([
    [1, 41.0, 49600.0, 1],
    [0, 46.0, 38000.0, 1],
    [0, 29.0, 21000.0, 0],
    [0, 21.0, 41700.0, 2],
    [1, 28.0, 26100.0, 0]
])

# Обратимая матрица
matrix = np.array([
    [2, 0, 0, 0],
    [0, 2, 0, 0],
    [0, 0, 2, 0],
    [0, 0, 0, 2]
])

# Умножение признаков на обратимую матрицу
transformed_features = np.dot(features, matrix)

print("Исходные признаки:\n", features)
print("\nУмноженные признаки:\n", transformed_features)

Исходные признаки:
 [[1.00e+00 4.10e+01 4.96e+04 1.00e+00]
 [0.00e+00 4.60e+01 3.80e+04 1.00e+00]
 [0.00e+00 2.90e+01 2.10e+04 0.00e+00]
 [0.00e+00 2.10e+01 4.17e+04 2.00e+00]
 [1.00e+00 2.80e+01 2.61e+04 0.00e+00]]

Умноженные признаки:
 [[2.00e+00 8.20e+01 9.92e+04 2.00e+00]
 [0.00e+00 9.20e+01 7.60e+04 2.00e+00]
 [0.00e+00 5.80e+01 4.20e+04 0.00e+00]
 [0.00e+00 4.20e+01 8.34e+04 4.00e+00]
 [2.00e+00 5.60e+01 5.22e+04 0.00e+00]]


In [5]:
# Целевой признак
target = np.array([0, 1, 0, 0, 0])

# Создание и обучение модели на исходных признаках
model_original = LinearRegression()
model_original.fit(features, target)

# Создание и обучение модели на преобразованных признаках
model_transformed = LinearRegression()
model_transformed.fit(transformed_features, target)

# Предсказание на исходных признаках
predictions_original = model_original.predict(features)
r2_original = r2_score(target, predictions_original)

# Предсказание на преобразованных признаках
predictions_transformed = model_transformed.predict(transformed_features)
r2_transformed = r2_score(target, predictions_transformed)

print("R2 модели на исходных признаках:", r2_original)
print("R2 модели на преобразованных признаках:", r2_transformed)

R2 модели на исходных признаках: 1.0
R2 модели на преобразованных признаках: 1.0


## Алгоритм преобразования

**Алгоритм**

Простой вариант для обеспечения защиты данных, одновременно сохраняя качество моделей машинного обучения, может быть умножение признаков на случайную обратимую матрицу.

Алгоритм преобразования данных с использованием умножения на обратимую матрицу:
1. Генерация случайной обратимой матрицы.
2. Умножение исходных признаков на полученную матрицу.

**Обоснование**

Такое преобразование позволит сделать исходные данные неузнаваемыми и сохранит их статистические характеристики. При этом, качество моделей машинного обучения сохранится, поскольку линейные модели, такие как линейная регрессия, инвариантны к линейным преобразованиям и сохранят свои предсказания с теми же ошибками.

## Проверка алгоритма

In [7]:
# Генерация случайной обратимой матрицы
matrix = np.random.rand(features.shape[1], features.shape[1])
while np.linalg.det(matrix) == 0:
    matrix = np.random.rand(features.shape[1], features.shape[1])
    
# Выделение признаков из представленного датасета
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

# Обучение модели lR
model = LinearRegression()
model.fit(features, target)

# Получение предсказаний на исходных данных
predictions = model.predict(features)
r2_original = r2_score(target, predictions)

# Умножение признаков на обратимую матрицу
transformed_features = np.dot(features, matrix)

# Создание и обучение модели после преобразования признаков
model_transformed = LinearRegression()
model_transformed.fit(transformed_features, target)

# Предсказание на преобразованных признаках
predictions_transformed = model_transformed.predict(transformed_features)
r2_transformed = r2_score(target, predictions_transformed)

print("R2 до преобразования признаков:", r2_original.round(3))
print("R2 после преобразования признаков:", r2_transformed.round(3))

R2 до преобразования признаков: 0.425
R2 после преобразования признаков: 0.425
